In [104]:
# import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

In [105]:
from reader import *
from string_matching_by_word import *
from string_matching_by_spaCy_NP import *
from word_embeddings_by_spaCy_NP import *
from hobbs import entry

In [106]:
import warnings
warnings.filterwarnings("ignore")

### Read Document

In [107]:
input_path = '/Users/Barretts Laptop/Documents/GitHub/nlp-project2019/dev/b3.input'
key_path = '/Users/Barretts Laptop/Documents/GitHub/nlp-project2019/dev/b3.key'

In [108]:
read = ReadInput(input_path)
list_of_sentences = read.getListOfSentences()
full_text = read.getText()

### Get Sentence Dict and Cluster Head Dict

In [109]:
sentence_dict = getSentenceDict(list_of_sentences)
cluster_head_dict = getClusterHeads(sentence_dict)

### Get Coref for exact match by word

In [110]:
coref_dict_match_word = getCorefDict_match_word(sentence_dict, cluster_head_dict,90)

### Get Coref for threshold match by NP

In [111]:
coref_dict_match_NP = getCorefDict_match_NP(sentence_dict, cluster_head_dict,80)

### Get Coref for Word Embedding Similarity by NP

In [112]:
coref_dict_all_sorted = getCorefDict_all_sorted(sentence_dict, cluster_head_dict,0.5)
coref_dict_max_sentence = getCorefDict_max_of_each_sentence(sentence_dict, cluster_head_dict,0.5)

In [113]:
coref_dict_all_sorted_top3 = get_TopN_Matches(coref_dict_all_sorted,3)

### Merging coref_dict_match_NP with coref_dict_all_sorted_top3

In [114]:
def mergeDicts(d1, d2):
    
    d1_keys = d1.keys()
    d2_keys = d2.keys()
    
    d_keys = list(set(d1_keys).union(set(d2_keys)))
    d = {}
    
    for key in d_keys:

        d1_val = []
        d2_val = []
        
        if(key in d1_keys):
            d1_val = set(tuple(i) for i in d1[key])
        if(key in d2_keys):
            d2_val = set(tuple(i) for i in d2[key])
            
        if(len(d1_val)!= 0 and len(d2_val)!= 0):
            d[key] = list(d1_val.union(d2_val))
            d[key] = [list(i) for i in d[key]]
        elif(len(d1_val)== 0 and len(d2_val)!= 0):
            d[key] = d2[key]
        elif(len(d1_val)!= 0 and len(d2_val)== 0):
            d[key] = d1[key]
    
    return d

In [115]:
coref_final = mergeDicts(coref_dict_match_NP, coref_dict_all_sorted)

In [119]:
coref_final

list

### Passing the reference dict to Hobbs

In [117]:
coref_final_with_pro = entry(list_of_sentences, cluster_head_dict, coref_final)

KeyError: 'X9'

In [ ]:
# coref_final_with_pro

In [ ]:
for cluster_id, cluster_val in cluster_head_dict.items():
    if(cluster_id not in coref_final_with_pro.keys()):
        coref_final_with_pro[cluster_id] = [['',int(cluster_val[0]),'', None]]

### Print Output

In [ ]:
def printOP(cluster_head_dict, coref_final_with_pro):
    
    for cluster_id, cluster_head_name in cluster_head_dict.items():

        print('<COREF ID="{}">{}</COREF>'.format(cluster_id, cluster_head_name[1]))

        coreferences = coref_final_with_pro[cluster_id]
        list_of_sent_ids = list(map(lambda x: x[1], coreferences))
        sorted_index_sent_ids = [i[0] for i in sorted(enumerate(list_of_sent_ids), key=lambda x:x[1])]
        coreferences = [coreferences[i] for i in sorted_index_sent_ids]

        for coref in coreferences:
            if(coref[0] == ''):
                continue
            print('{{{0}}}'.format(coref[1]) + ' ' + '{' + coref[2] + '}')
        print('\n', end = '')      

In [ ]:
%%capture cap --no-stderr
printOP(cluster_head_dict, coref_final_with_pro)

In [ ]:
with open('b3.response','w') as f:
    f.write(cap.stdout)

### Answer Key

In [103]:
# reader_o = ReadInput(key_path)
# ans = reader_o.getListOfSentences()
# ans